In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import json
import matplotlib.pyplot as plt
from IPython.display import display
import os
import numpy as np
import sys 
sys.path.append('../../modules')
import eumf_data as data
import eumf_eval as eval
from sklearn import preprocessing
from sklearn import pipeline
from sklearn import dummy
from sklearn import compose
from sklearn import multioutput
from sklearn import linear_model
from sklearn import ensemble
from sklearn import feature_selection
from sklearn import model_selection

pd.set_option('display.min_rows', 20)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)


## Load Data

In [7]:
values = data.load_migration_rates_from_csv()
trends = data.load_trends_from_csv()
countries = data.get_countries()
keyword_ids = trends.columns.levels[0].tolist()


# panel = pd.concat([values, trends], axis='columns')
panel = values.join(trends, how="outer")



## Models

In [21]:
t_min = "2012-01-01"
t_max = "2019-12-01"
t_split_lower = "2017-12-01"
t_split_upper = "2018-01-01"

panel_resampled = panel.resample("3M").mean()
panel_lags = data.create_lags(
    panel_resampled, lags=[1, 2, 3, 4], columns=["19", "value"]
).fillna(0.0)

x = panel_lags
# x.columns = ["_".join(col) for col in x.columns.values]  # flatten columns from x
y = panel_resampled["value"]

# transformation: differencing + stacking
x_trans = np.log(x+1) - np.log(x.shift(4)+1)
y_trans = np.log(y+10) - np.log(y.shift(4)+10)

x_trans_stack = x_trans.stack().reset_index(level=1)
y_trans_stack = y_trans.stack().droplevel(level=1)

x_train = x_trans_stack[t_min:t_split_lower]
y_train = y_trans_stack[t_min:t_split_lower]
x_test = x_trans_stack[t_split_upper:t_max]
y_test = y_trans_stack[t_split_upper:t_max]

# cv = model_selection.TimeSeriesSplit(n_splits=5, test_size=4)
cv = model_selection.KFold(n_splits=6)

ct = compose.make_column_transformer(
    (preprocessing.OneHotEncoder(), ["country"]),
    remainder="passthrough",
    sparse_threshold=0,
)

reg = pipeline.make_pipeline(ct, ensemble.RandomForestRegressor(random_state=0),)

hptuner = model_selection.GridSearchCV(
    reg,
    {
        "randomforestregressor__min_samples_split": [2],
        "randomforestregressor__max_depth": [8],
        "randomforestregressor__ccp_alpha": [0.0, 0.01, 0.05],
        "randomforestregressor__min_samples_leaf": [4],
        "randomforestregressor__max_features": ["auto", "sqrt", "log2"]
    },
    cv=cv,
    scoring=eval.scorer_rmse,
    n_jobs=-1,
)
hptuner.fit(x_train, y_train)
reg = hptuner.best_estimator_
display(hptuner.best_params_)

cv_score = eval.score_cv(reg, x_train, y_train, cv=cv)
test_score = eval.score_test(reg, x_test, y_test)

print("Mean CV score:")
display(cv_score.mean())

print("OOS score:")
display(test_score)


{'randomforestregressor__ccp_alpha': 0.0,
 'randomforestregressor__max_depth': 8,
 'randomforestregressor__max_features': 'auto',
 'randomforestregressor__min_samples_leaf': 4,
 'randomforestregressor__min_samples_split': 2}

Mean CV score:


fit_time                   0.197
score_time                 0.009
test_mae                  -0.076
test_rmse                 -0.108
test_explained_variance    0.454
test_r2                    0.427
dtype: float64

OOS score:


mae                  -0.092
rmse                 -0.176
explained_variance    0.196
r2                    0.157
dtype: float64

In [22]:
pd.Series(reg.steps[-1][1].feature_importances_[-8:], index=x_trans_stack.columns[1:])

19_1       0.031
19_2       0.019
19_3       0.030
19_4       0.023
value_1    0.772
value_2    0.043
value_3    0.028
value_4    0.043
dtype: float64

In [25]:
pred_arr = reg.predict(x_trans_stack[t_min:t_max])
y_pred_trans = pd.Series(pred_arr, index=y[t_min:t_max].stack().index).unstack()
y_pred = np.exp(y_pred_trans) * (y.shift(4) + 10) - 10

#y_pred_trans_cv
# y_trans



In [24]:
country_names = {
    "FR": "Frankreich",
    "GB": "Vereinigtes Königreich",
    "IT": "Italien",
    "ES": "Spanien",
    "PL": "Polen",
    "RO": "Rumänien",
    "NL": "Niederlande",
    "BE": "Belgien",
    "GR": "Griechenland",
    "CZ": "Tschechien",
    "PT": "Portugal",
    "SE": "Schweden",
    "HU": "Ungarn",
    "AT": "Österreich",
    "CH": "Schweiz",
    "BG": "Bulgarien",
    "DK": "Dänemark",
    "FI": "Finnland",
    "SK": "Slowakei",
    "IE": "Irland",
    "HR": "Kroatien",
    "LT": "Lettland",
    "SI": "Slowenien",
    "LV": "Litauen",
    "EE": "Estland",
    "CY": "Zypern",
    "LU": "Luxemburg",
}

In [26]:
(
    pd.concat(
        {
            "Anmeldungen in Deutschland": y_trans["2011-01-01":t_max],
            "Vorhersage": y_pred_trans[t_min:t_max],
        },
        axis=1,
    )
    .stack()
    .loc[pd.IndexSlice[:, ["IT", "ES", "PT", "GR"]],]
    .rename(country_names)
).to_csv("data/mvp/forecast_suedeuropa_transformiert.csv")



In [35]:
(
    (
        pd.concat(
            {
                "Anmeldungen in Deutschland": y["2011-01-01":t_max]/y["2010-01-01":"2010-12-01"].mean() * 100,
                "Vorhersage": y_pred[t_min:t_max]/y["2010-01-01":"2010-12-01"].mean() * 100,
            },
            axis=1,
        )
        # / y["2010-01-01":"2010-12-31"]
        # * 100
    )
    .stack()
    .loc[pd.IndexSlice[:, ["IT", "ES", "PT", "GR"]],]
    .rename(country_names)
).to_csv("data/mvp/forecast_suedeuropa.csv")



In [28]:
t_min = "2012-01-01"
t_max = "2019-12-01"
t_split_lower = "2017-12-01"
t_split_upper = "2018-01-01"

panel_resampled = panel.resample("3M").mean()
panel_lags = data.create_lags(
    panel_resampled, lags=[1, 2, 3, 4], columns=["value"]
).fillna(0.0)

x = panel_lags
# x.columns = ["_".join(col) for col in x.columns.values]  # flatten columns from x
y = panel_resampled["value"]

# transformation: differencing + stacking
x_trans = np.log(x+1) - np.log(x.shift(4)+1)
y_trans = np.log(y+10) - np.log(y.shift(4)+10)

x_trans_stack = x_trans.stack().reset_index(level=1)
y_trans_stack = y_trans.stack().droplevel(level=1)

x_train = x_trans_stack[t_min:t_split_lower]
y_train = y_trans_stack[t_min:t_split_lower]
x_test = x_trans_stack[t_split_upper:t_max]
y_test = y_trans_stack[t_split_upper:t_max]

# cv = model_selection.TimeSeriesSplit(n_splits=5, test_size=4)
cv = model_selection.KFold(n_splits=6)

ct = compose.make_column_transformer(
    (preprocessing.OneHotEncoder(), ["country"]),
    remainder="passthrough",
    sparse_threshold=0,
)

reg = pipeline.make_pipeline(ct, ensemble.RandomForestRegressor(random_state=0),)

hptuner = model_selection.GridSearchCV(
    reg,
    {
        "randomforestregressor__min_samples_split": [2],
        "randomforestregressor__max_depth": [8],
        "randomforestregressor__ccp_alpha": [0.0, 0.01, 0.05],
        "randomforestregressor__min_samples_leaf": [4],
        "randomforestregressor__max_features": ["auto", "sqrt", "log2"]
    },
    cv=cv,
    scoring=eval.scorer_rmse,
    n_jobs=-1,
)
hptuner.fit(x_train, y_train)
reg = hptuner.best_estimator_
display(hptuner.best_params_)

cv_score = eval.score_cv(reg, x_train, y_train, cv=cv)
test_score = eval.score_test(reg, x_test, y_test)

print("Mean CV score:")
display(cv_score.mean())

print("OOS score:")
display(test_score)


{'randomforestregressor__ccp_alpha': 0.0,
 'randomforestregressor__max_depth': 8,
 'randomforestregressor__max_features': 'auto',
 'randomforestregressor__min_samples_leaf': 4,
 'randomforestregressor__min_samples_split': 2}

Mean CV score:


fit_time                   0.160
score_time                 0.010
test_mae                  -0.079
test_rmse                 -0.110
test_explained_variance    0.435
test_r2                    0.399
dtype: float64

OOS score:


mae                  -0.091
rmse                 -0.176
explained_variance    0.195
r2                    0.155
dtype: float64

In [29]:
pred_arr_2 = reg.predict(x_trans_stack[t_min:t_max])
y_pred_trans_2 = pd.Series(pred_arr_2, index=y[t_min:t_max].stack().index).unstack()
# y_pred = np.exp(y_pred_trans) * (y.shift(4) + 10) - 10

#y_pred_trans_cv
# y_trans



In [31]:
(
    pd.concat(
        {
            "Anmeldungen in Deutschland": y_trans["2011-01-01":t_max],
            "Vorhersage": y_pred_trans_2[t_min:t_max],
        },
        axis=1,
    )
    .stack()
    .loc[pd.IndexSlice[:, ["IT", "ES", "PT", "GR"]],]
    .rename(country_names)
).to_csv("data/mvp/forecast_suedeuropa_nogoogle_transformiert.csv")



In [30]:
(
    pd.concat(
        {
            "Anmeldungen in Deutschland": y_trans["2011-01-01":t_max],
            "Vorhersage (ohne Google Trends)": y_pred_trans_2[t_min:t_max],
            "Vorhersage (mit Google Trends)": y_pred_trans[t_min:t_max],
        },
        axis=1,
    )
    .stack()
    .loc[pd.IndexSlice[:, ["IT", "ES", "PT", "GR"]],]
    .rename(country_names)
).to_csv("data/mvp/forecast_suedeuropa_beide_transformiert.csv")

